<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-2-Opti.-Mat./blob/main/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 35.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from gurobipy import*

In [3]:
#para esta pregunta se nos dan ciertos coeficientes a los cuales se les asignaran un valor
#Primero voy a plantear nuevamente el problema de optimización

#Definimos los datos que nos entrega el problema.
betha = [0,100,120,130,140,150,200,0] #Esto representa el costo de almacenaje



c = np.array([[0,10,25,30,0,0,20,0],[0,0,0,0,15,15,0,0],[0,0,0,0,0,15,0,0],
 [0,0,0,0,0,0,10,0],[0,0,0,0,0,0,0,25],[0,0,0,0,0,0,0,80],[0,0,0,0,0,0,0,25],  #representa el costo de ir desdela bodega i-esima hasta la j-esima, cij = 0 representa que esa ruta no existe
  [0,0,0,0,0,0,0,0]])

r = np.array([[0,500,350,500,0,0,600,0],[0,0,0,0,250,350,0,0],[0,0,0,0,0,350,0,0],
 [0,0,0,0,0,0,500,0],[0,0,0,0,0,0,0,800],[0,0,0,0,0,0,0,500],[0,0,0,0,0,0,0,500], #Representa cuanto se piuede transportar en la ruta desde i hasta j. Rij = 0 significa que esa ruta no está definida
  [0,0,0,0,0,0,0,0]])



In [4]:
#planteamos el problema de optimización
m = Model('Penalización')
n = 8

#definimos las variables, en este caso tenemos un problema de una sola variable, el cual no es lineal

x = m.addVars(n,n,vtype=GRB.CONTINUOUS, name = 'Cantidad_de_elemntos_por_bodega')
#restricciones:

#Con esta restriccion dejamos que el la primera columna tenga sea el inicio
m.addConstr((quicksum(x[0,j] for j in range(n))) == 1000, name='Inicio')

#En el final deben existir 1000 unidades
m.addConstr(quicksum(x[i, 7] for i in range(n)) == 1000, name='Llegada')

#se deben bloquear las rutas no existentes
for i in range(n):
    for j in range(n):
        if float(r[i, j]) == 0:  # Bloquear rutas no válidas
            m.addConstr(x[i, j] == 0, name=f'Bloqueo_{i}_{j}')
        else:
            m.addConstr(x[i, j] <= r[i, j], name=f'Capacidad_{i}_{j}')
#hay que asegurar que lo que entra es igual a lo que sale
for i in range(1, 7):
    entrada = quicksum(x[j, i] for j in range(n))
    salida = quicksum(x[i, j] for j in range(n))
    m.addConstr(entrada == salida, name=f'balance_{i}')

m.setObjective(quicksum(float(c[i, j]) * x[i, j] * x[i, j] for i in range(n) for j in range(n)) +
    quicksum(float(betha[i]) * quicksum(x[i, j] for j in range(n)) for i in range(n)),GRB.MINIMIZE)

m.optimize()

print(f"Status: {m.Status}")
if m.Status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            val = x[i, j].X
            if val > 1e-6:
                print(f"x[{i},{j}] = {val:.2f}")
else:
    print("No se encontró solución óptima.")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 72 rows, 64 columns and 164 nonzeros
Model fingerprint: 0x9a70293f
Model has 11 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 65 rows and 53 columns
Presolve time: 0.01s
Presolved: 7 rows, 11 columns, 18 nonzeros
Presolved model has 11 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.000e+00
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
It

In [50]:
# Definir rutas válidas
rutas_validas = [(i,j) for i in range(n) for j in range(n) if r[i,j] > 0]

# Crear modelo y variables
m2 = Model('modelo_penalizacion')
x = m2.addVars(rutas_validas, vtype=GRB.CONTINUOUS, name='Flujo')

# Función objetivo considerando solo rutas válidas
f = (quicksum(x[i,j] * x[i,j] for (i,j) in rutas_validas) + \
     quicksum(float(betha[i]) * quicksum(x[i,j] for j in range(n) if (i,j) in rutas_validas)
             for i in range(n)))

# Restricciones modificadas para usar solo rutas válidas
h1 = quicksum(x[0,j] for j in range(n) if (0,j) in rutas_validas) - 1000
h2 = quicksum(x[i,7] for i in range(n) if (i,7) in rutas_validas) - 1000

# Restricciones de conservación de flujo CORREGIDAS
h4 = []
for i in range(1, 7):  # Nodos intermedios
    entrada = quicksum(x[j, i] for j in range(n) if (j,i) in rutas_validas)
    salida = quicksum(x[i, j] for j in range(n) if (i,j) in rutas_validas)
    h4.append((entrada - salida)**2)

# Función objetivo con penalización
k = 0
mu_k = 10**k
phi = f + mu_k*(h1**2 + h2**2 + quicksum(h4))

# Optimización
m2.setObjective(phi, GRB.MINIMIZE)
m2.optimize()

# Resultados CORREGIDOS
print(f"Status: {m2.status}")  # Usar .status en minúscula
if m2.status == GRB.OPTIMAL:
    for (i,j) in rutas_validas:
        val = x[i,j].X
        if val > 1e-6:
            print(f"x[{i},{j}] = {val:.6f}")
else:
    print("No se encontró solución óptima.")
    print(f"Código de estado: {m2.status}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 11 columns and 0 nonzeros
Model fingerprint: 0xd77c26c5
Model has 32 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+02, 2e+03]
  QObjective range [4e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 11 columns, 0 nonzeros
Presolved model has 32 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 10
 AA' NZ     : 4.300e+01
 Factor NZ  : 5.500e+01
 Factor Ops : 3.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.22507633e+07 -2.

In [53]:
from gurobipy import Model, GRB, quicksum

# Parámetros del algoritmo
mu_k = 1.0       # μ_0
epsilon = 1e-5   # ε
beta = 10        # β
K_max = 10       # K
k = 0            # Contador de iteraciones

# Datos del problema (ejemplo)
betha = [0,100,120,130,140,150,200,0]
n = len(betha)

# Rutas válidas
rutas_validas = [(i,j) for i in range(n) for j in range(n) if r[i,j] > 0]

# Algoritmo de penalización cuadrática
while k <= K_max:
    # Crear modelo para esta iteración
    m2 = Model('penalizacion_cuadratica')
    m2.Params.NonConvex = 2  # Para problemas no convexos

    # Variables con restricciones 0 ≤ x ≤ Q (r[i,j] en nuestro caso)
    x = m2.addVars(rutas_validas, vtype=GRB.CONTINUOUS, name='Flujo')
    for (i,j) in rutas_validas:
        x[i,j].LB = 0
        x[i,j].UB = r[i,j]

    # Función f(x)
    f = (quicksum(x[i,j] * x[i,j] for (i,j) in rutas_validas) + \
         quicksum(float(betha[i]) * quicksum(x[i,j] for j in range(n) if (i,j) in rutas_validas)
                 for i in range(n)))

    # Restricciones h_j(x)
    h1 = quicksum(x[0,j] for j in range(n) if (0,j) in rutas_validas) - 1000  # Flujo saliente nodo 0
    h2 = quicksum(x[i,7] for i in range(n) if (i,7) in rutas_validas) - 1000  # Flujo entrante nodo 7

    # Restricciones de conservación de flujo (nodos intermedios)
    h_conservacion = []
    for i in range(1, n-1):  # Todos los nodos excepto origen (0) y destino (7)
        entrada = quicksum(x[j,i] for j in range(n) if (j,i) in rutas_validas)
        salida = quicksum(x[i,j] for j in range(n) if (i,j) in rutas_validas)
        h_conservacion.append((entrada - salida)**2)

    # Función phi(x) = f(x) + μ_k Σ h_j(x)^2
    phi = f + mu_k * (h1**2 + h2**2 + quicksum(h_conservacion))

    # Resolver el problema
    m2.setObjective(phi, GRB.MINIMIZE)
    m2.optimize()

    # Verificar criterio de parada
    sum_h_squared = (h1.getValue()**2 + h2.getValue()**2 +
                     sum(h.getValue()**2 for h in h_conservacion))

    print(f"Iteración {k}: μ_k = {mu_k:.1e}, Σh_j^2 = {sum_h_squared:.6e}")

    # Criterio de parada
    if mu_k * sum_h_squared < epsilon or k == K_max:
        print("\nSolución encontrada:")
        if m2.status == GRB.OPTIMAL:
            for (i,j) in rutas_validas:
                val = x[i,j].X
                if val > 1e-6:
                    print(f"x[{i},{j}] = {val:.6f}")
        else:
            print("No se encontró solución óptima.")
        break

    # Actualizar para siguiente iteración
    mu_k *= beta
    k += 1

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
NonConvex  2

Optimize a model with 0 rows, 11 columns and 0 nonzeros
Model fingerprint: 0x72b7bdad
Model has 32 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+02, 2e+03]
  QObjective range [4e+00, 6e+00]
  Bounds range     [2e+02, 8e+02]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 11 columns, 0 nonzeros
Presolved model has 32 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 10
 AA' NZ     : 4.300e+01
 Factor NZ  : 5.500e+01
 Factor Ops : 3.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal         